In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D  # Needed for 3D projection
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.cluster import KMeans
from PIL import Image
import umap
import os
from itertools import product


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
def downsample(img, block_size, offset):
    h, w, c = img.shape
    bh, bw = block_size
    oy, ox = offset

    # Compute the number of blocks that fit after applying the offset
    nh = (h - oy) // bh
    nw = (w - ox) // bw

    # Crop the image to fit full blocks
    img_crop = img[oy:oy+nh*bh, ox:ox+nw*bw]

    # Reshape and take mean over blocks
    reshaped = img_crop.reshape(nh, bh, nw, bw, c)
    low_res = reshaped.mean(axis=(1, 3))  # shape: (nh, nw, c)

    return low_res

def upsample(low_res, block_size, img_shape):
    nh, nw, c = low_res.shape
    bh, bw = block_size
    upsampled = np.kron(low_res, np.ones((bh, bw, 1)))  # Nearest-neighbor upscale
    return upsampled[:img_shape[0], :img_shape[1]]  # Crop if needed

def compute_error(original, upsampled):
    diff = original - upsampled
    error = np.sqrt((diff ** 2).sum(axis=2))  # Per-pixel Euclidean distance
    return error.mean()

def find_best_parameters(img, block_min: int = 8, block_max: int = 16):
    block_range = (block_min, block_max)
    max_offset = block_max
    
    h, w, _ = img.shape
    best_error = float('inf')
    best_params = None

    for block_size in range(block_range[0], block_range[1] + 1):
        error_mat = np.nan * np.ones([max_offset, max_offset])
        print(f'Now testing block size: {block_size}')
        for oy in range(0, max_offset + 1):
            for ox in range(0, max_offset + 1):
                try:
                    low_res = downsample(img, (block_size, block_size), (oy, ox))
                    upsampled = upsample(low_res, (block_size, block_size), (h, w))
                    error = compute_error(img.astype(np.float32), upsampled.astype(np.float32))

                    if error < best_error:
                        best_error = error
                        best_params = (block_size, ox, oy)

                    error_mat[ox, oy] = error
                except Exception as e:
                    # Might happen on edge cases where crop is too small, skip them
                    continue

            plt.figure()
            sns.heatmap(error_mat)
            plt.title(f'error offset matrix for block_size: {block_size}')
            plt.xlabel('x offset (pixels)')
            plt.ylabel('y offset (pixels)')

    return best_params, best_error

In [ ]:
# --- Load Image ---
base_dir = os.path.abspath(os.getcwd())
file_name = 'ent_mini_boss_1024px.png'
input_path = os.path.join(base_dir, 'projects', 'castle', file_name)
img_rgba = Image.open(input_path).convert("RGBA")
img_data = np.array(img_rgba)


In [ ]:
%%time

# manually set params after inspecting approximate block size from image
block_min = 9
block_max = 14

# --- Find best parameters ---  # NOTE: this takes about ~30 s per block_size for a 1024x1024 image
best_params, best_error = find_best_parameters(img_data, block_min, block_max)
print(f"Best block size & offset: {best_params}, Error: {best_error:.2f}")